In [4]:
import torch
import numpy as np
import sys
import torchvision
from torchvision import transforms
from torch.nn import init
from torch import nn

In [3]:
num_inputs,num_outputs,num_hiddens=784,10,256

In [21]:
net=nn.Sequential(
    FlattenLayer(),
    nn.Linear(num_inputs,num_hiddens),
    nn.ReLU(),
    nn.Linear(num_hiddens,num_outputs),
)

In [22]:
for params in net.parameters():
    init.normal_(params, mean=0, std=0.01)

In [23]:
batch_size=256
mnist_train=torchvision.datasets.FashionMNIST(
    root='F:\study\ml\DataSet\FashionMNIST',train=True,
    download=True,transform=transforms.ToTensor())
mnist_test=torchvision.datasets.FashionMNIST(
    root='F:\study\ml\DataSet\FashionMNIST',train=False,
    download=True,transform=transforms.ToTensor())

In [24]:
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,
                                      shuffle=True,num_workers=4)
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,
                                      shuffle=True,num_workers=4)

In [29]:
num_epochs, lr = 5, 0.1


def train_ch3(net,
              train_iter,
              test_iter,
              loss,
              num_epochs,
              batch_size,
              params=None,
              lr=None,
              optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for x, y in train_iter:
            y_hat = net(x)
            l = loss(y_hat, y).sum()
#             print('-----------')
#             print('y_hat: \n',y_hat)
#             print('y: \n',y)
#             print(l.item())
#             print('-----------')
            
            if params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
#             if optimizer is None:
#             sgd(x, lr, params)
            optimizer.step()

            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).float().sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d ,loss %.4f,train acc %.3f,test acc %.3f' %
              (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

In [30]:
class FlattenLayer(nn.Module):
	def __init__(self):
		super(FlattenLayer, self).__init__()
	def forward(self, x): # x shape: (batch, *, *, ...)
		return x.view(x.shape[0], -1)

In [32]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for x, y in data_iter:
        acc_sum += (net(x).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [33]:
batch_size = 256

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
num_epochs = 5
train_ch3(net, train_iter, test_iter, loss, num_epochs,
batch_size, None, None, optimizer)

epoch 1 ,loss 0.0188,train acc 0.140,test acc 0.142
epoch 2 ,loss 0.0181,train acc 0.111,test acc 0.100
epoch 3 ,loss 0.0174,train acc 0.105,test acc 0.103
epoch 4 ,loss 0.0185,train acc 0.103,test acc 0.103
epoch 5 ,loss 0.0186,train acc 0.106,test acc 0.103


In [ ]:
np.convolve()